# Improved Uncertain Activity Bandits

Improve Uncertain (non-Oracle) Activity Bandits (from their baseline) with unknown transition functions. We improve these by imporving the way they learn the transition function using a normal approximation (Wilson score) along with fixing some bugs in their code

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
from scipy.stats import norm

In [6]:
from rmab.simulator import RMABSimulator, random_valid_transition
from rmab.uc_whittle import UCWhittleBuggy, UCWhittle, UCWhittleOracle 

from rmab.baselines import optimal_whittle, random_policy, WIQL
from rmab.fr_dynamics import get_all_transitions
from rmab.utils import get_save_path, delete_duplicate_results


In [7]:
is_jupyter = 'ipykernel' in sys.modules

In [8]:
if is_jupyter: 
    seed        = 42
    n_arms      = 8
    budget      = 3
    discount    = 0.9
    alpha       = 3 
    n_episodes  = 30
    episode_len = 20
    n_epochs    = 10
    save_name = 'normal_confidence'
    save_with_date = False 
    dataset = 'fr'
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_arms',         '-N', help='num beneficiaries (arms)', type=int, default=8)
    parser.add_argument('--episode_len',    '-H', help='episode length', type=int, default=20)
    parser.add_argument('--n_episodes',     '-T', help='num episodes', type=int, default=30)
    parser.add_argument('--budget',         '-B', help='budget', type=int, default=3)
    parser.add_argument('--n_epochs',       '-E', help='number of epochs (num_repeats)', type=int, default=10)
    parser.add_argument('--discount',       '-d', help='discount factor', type=float, default=0.9)
    parser.add_argument('--alpha',          '-a', help='alpha: for conf radius', type=float, default=3)
    parser.add_argument('--seed',           '-s', help='random seed', type=int, default=42)
    parser.add_argument('--save_name',      '-n', help='save name', type=str, default='results')
    parser.add_argument('--dataset',      '-ds', help='which dataset', type=str, default='fr')
    parser.add_argument('--use_date', action='store_true')

    args = parser.parse_args()

    n_arms      = args.n_arms
    budget      = args.budget
    discount    = args.discount
    alpha       = args.alpha 
    seed        = args.seed
    n_episodes  = args.n_episodes
    episode_len = args.episode_len
    n_epochs    = args.n_epochs
    save_name   = args.save_name 
    save_with_date = args.use_date 
    dataset = args.dataset 



In [9]:
n_states = 2
n_actions = 2

In [10]:
all_population_size = 100 # number of random arms to generate

if dataset == 'fr':
    all_transitions = get_all_transitions(all_population_size)
elif dataset == 'synthetic':
    all_transitions = random_valid_transition(all_population_size, n_states, n_actions)
else:
    raise Exception("Dataset {} is not found".format(dataset))

In [11]:
all_features = np.arange(all_population_size)

In [12]:
np.random.seed(seed)
random.seed(seed)
simulator = RMABSimulator(all_population_size, all_features, all_transitions,
            n_arms, episode_len, n_epochs, n_episodes, budget, number_states=n_states)

acting should always be good! 0.000 < 0.044
acting should always be good! 0.000 < 0.162
acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [83 53 70 45 44 39 22 80]
cohort [45 80  5 29 78 35 86 14]
cohort [42 28 84 76 68 46 97 15]
cohort [51 29 14 74 15  3 31 87]
cohort [55  0 94  4 72 26 38  3]
cohort [69 66 55 63 30 61 42 29]
cohort [83 68 66 27 99 32 35 23]
cohort [31  1 98 11 36  4 84 58]
cohort [ 5 41 86 49 97 13 58 42]
cohort [22 71 78 83 30 48 34 29]


In [13]:
np.random.seed(seed)
random.seed(seed)
rewards_without_norm = UCWhittle(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB',norm_confidence=False)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
---------------------------------------------------
0 0  | a  [0 0 0 0 0 1 1 1]  | s'  [0 0 1 0 0 1 0 1]  | r  7    | WI  [-0. -0. -0. -0. -0. -0. -0. -0.]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 0 0 1 1 1]  | s'  [0 0 0 0 0 1 0 0]  | r  1    | WI  [-0. -0. -0. -0. -0. -0. -0. -0.]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 0 0 0 0 1 1 1]  | s'  [1 1 1 0 0 1 1 0]  | r  3    | WI  [-0. -0. -0. -0. -0. -0. -0. -0.]
instance 0, ep 11, state [1 0 1 1 0 1 1 1

In [14]:
np.random.seed(seed)
random.seed(seed)
rewards_with_norm = UCWhittle(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB',norm_confidence=True)
np.mean(rewards_with_norm)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
---------------------------------------------------
0 0  | a  [0 0 0 0 0 1 1 1]  | s'  [0 0 1 0 0 1 0 1]  | r  7    | WI  [-0. -0. -0. -0. -0. -0. -0. -0.]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 1 1 0 0 0 0 1]  | s'  [0 0 0 0 0 0 0 1]  | r  1    | WI  [  0.283   0.648   0.578   0.398 -10.      0.422   0.47    0.55 ]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [1 0 0 1 0 0 0 1]  | s'  [1 0 1 1 0 0 0 0]  | r  3    | WI  [  0.639   0.505   0.207   0.54  -10.    -1

3.5045

In [15]:
np.random.seed(seed)
random.seed(seed)
perfect_rewards = UCWhittleOracle(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB',norm_confidence=True)
np.mean(perfect_rewards)

solving UCWhittle using method: UCB
first state [0 1 1 1 1 1 1 1]
---------------------------------------------------
0 0  | a  [0 0 1 1 0 0 0 1]  | s'  [1 0 1 1 1 0 1 1]  | r  7    | WI  [-10.      0.163   0.251   0.172 -10.    -10.    -10.      0.239]
instance 0, ep 1, state [0 0 0 1 1 1 0 1]
instance 0, ep 2, state [0 1 1 1 0 0 0 1]
instance 0, ep 3, state [0 0 0 1 0 1 1 0]
instance 0, ep 4, state [1 1 0 1 1 1 1 1]
instance 0, ep 5, state [0 0 0 0 0 1 0 0]
---------------------------------------------------
0 100  | a  [0 0 1 1 0 0 0 1]  | s'  [0 0 0 1 0 0 0 1]  | r  1    | WI  [  0.292   0.543   0.58    0.737 -10.      0.322   0.498   1.   ]
instance 0, ep 6, state [0 1 0 1 0 0 0 0]
instance 0, ep 7, state [1 0 1 0 0 0 0 1]
instance 0, ep 8, state [0 0 0 0 1 1 0 1]
instance 0, ep 9, state [0 1 0 0 0 1 1 1]
instance 0, ep 10, state [0 1 1 0 0 0 1 0]
---------------------------------------------------
0 200  | a  [0 0 1 1 0 0 0 1]  | s'  [1 0 0 1 0 0 0 0]  | r  3    | WI  [  0.389   

3.5568333333333335

In [16]:
np.random.seed(seed)
random.seed(seed)
optimal_reward = optimal_whittle(simulator, n_episodes, n_epochs, discount)
np.mean(optimal_reward)

first state [0 1 1 1 1 1 1 1]
   state [0 1 1 1 1 1 1 1] state_WI [  0.39   0.47   0.47   0.26   0.45 -10.   -10.   -10.  ] sorted [2 1 4]
   state [0 1 1 0 1 0 1 1] state_WI [  0.39   0.47   0.47   0.71   0.45   0.52 -10.   -10.  ] sorted [3 5 2]
   state [0 1 1 1 1 1 0 0] state_WI [  0.39   0.47   0.47   0.26   0.45 -10.     0.49   0.82] sorted [7 6 2]
   state [0 1 1 1 0 0 0 0] state_WI [0.39 0.47 0.47 0.26 0.45 0.52 0.49 0.82] sorted [7 5 6]
   state [0 1 1 0 0 1 0 1] state_WI [  0.39   0.47   0.47   0.71   0.45 -10.     0.49 -10.  ] sorted [3 6 2]
   state [1 0 1 1 0 0 1 0] state_WI [  0.26   0.54   0.47   0.26   0.45   0.52 -10.     0.82] sorted [7 1 5]
   state [0 1 0 0 0 1 1 0] state_WI [  0.39   0.47   0.58   0.71   0.45 -10.   -10.     0.82] sorted [7 3 2]
   state [1 0 0 1 0 1 0 1] state_WI [  0.26   0.54   0.58   0.26   0.45 -10.     0.49 -10.  ] sorted [2 1 6]
   state [0 1 1 1 0 0 1 1] state_WI [  0.39   0.47   0.47   0.26   0.45   0.52 -10.   -10.  ] sorted [5 2 1]
   st

3.5778333333333334

In [17]:
data = {
    'mean_reward_baseline': np.mean(rewards_without_norm), 
    'mean_reward_norm': np.mean(rewards_with_norm), 
    'mean_reward_perfect': np.mean(perfect_rewards),
    'mean_optimal_reward': np.mean(optimal_reward),
    'parameters': 
        {'seed'      : seed,
        'n_arms'    : n_arms,
        'budget'    : budget,
        'discount'  : discount, 
        'alpha'     : alpha, 
        'n_episodes': n_episodes, 
        'episode_len': episode_len, 
        'n_epochs'  : n_epochs, 
        'dataset': dataset} 
}

In [68]:
save_path = get_save_path('better_bandit',save_name,seed,use_date=save_with_date)

In [69]:
delete_duplicate_results('better_bandit',save_name,data)

In [70]:
json.dump(data,open('../results/'+save_path,'w'))